In [1]:
import pandas as pd
import fiona
import geopandas as gpd
from sqlalchemy import create_engine
import numpy as np
#import dedupe
from dotenv import load_dotenv
import os
import pyArango
import usaddress
from usaddress import tag
from scourgify import normalize_address_record
import re
import json

In [2]:
from pyArango.connection import *
conn = Connection(username="root", password="0505")
corp_db = conn["corp_data"]

In [3]:
load_dotenv()
PG_CONNECT = os.getenv("PG_CONNECT")

In [6]:
# extracting the keys of the l3_assess collection
key_aql = "FOR x IN l3_assess RETURN x._key"
key_query_result = corp_db.AQLQuery(key_aql,rawResults=True, batchSize = 1000)
keys = []

for key in key_query_result:
    keys.append(key)

In [14]:
for i in keys[0:20]:
    prop = corp_db["l3_assess"].fetchDocument(i)
    print(corp_db["geo2corp"].getEdges(prop))

[]
[ArangoEdge '_id: geo2corp/30431374, _key: 30431374, _rev: _eb_OKfW---, _to: corp_data_processed/17794168, _from: l3_assess/28345022': <store: {'type': 'geo2corp/address'}>, ArangoEdge '_id: geo2corp/30431376, _key: 30431376, _rev: _eb_OKfa---, _to: corp_data_processed/18038480, _from: l3_assess/28345022': <store: {'type': 'geo2corp/address'}>, ArangoEdge '_id: geo2corp/30431378, _key: 30431378, _rev: _eb_OKfe---, _to: corp_data_processed/17943214, _from: l3_assess/28345022': <store: {'type': 'geo2corp/address'}>, ArangoEdge '_id: geo2corp/30431380, _key: 30431380, _rev: _eb_OKfm---, _to: corp_data_processed/17844839, _from: l3_assess/28345022': <store: {'type': 'geo2corp/address'}>, ArangoEdge '_id: geo2corp/30431382, _key: 30431382, _rev: _eb_OKfq---, _to: corp_data_processed/17885222, _from: l3_assess/28345022': <store: {'type': 'geo2corp/address'}>, ArangoEdge '_id: geo2corp/30431384, _key: 30431384, _rev: _eb_OKfy---, _to: corp_data_processed/17810670, _from: l3_assess/28345022

In [22]:
# extracting the keys of the l3_assess collection
key_aql = "FOR x IN corp_data_processed RETURN x._key"
key_query_result = corp_db.AQLQuery(key_aql,rawResults=True, batchSize = 1000)
corp_keys = []

for key in key_query_result:
    corp_keys.append(key)

In [62]:
for i in corp_keys[60:70]:
    corp = corp_db["corp_data_processed"].fetchDocument(i)
    edges = corp_db["corp2indiv"].getEdges(corp)
    print("corp", corp)
    print("\n")
    indivs = []
    shells = []
    
    for edge in edges:
        ind = edge._to[21:]
        indiv = corp_db["indiv_data_processed"].fetchDocument(ind)
        indivs.append(indiv)
        
        for indiv_edge in indiv_edges:
            shell_ind = indiv_edge._from[20:]
            shell = corp_db["corp_data_processed"].fetchDocument(shell_ind)
            if not in_vertices(shell,shells):
                shells.append(shell)
             
    
    print("shells", shells)
    print("\n")

corp ArangoDoc '_id: corp_data_processed/17778062, _key: 17778062, _rev: _eZCy3Xa--E': <store: {'DataID': '00v62r', 'EntityTypeDescriptor': 'Domestic Profit Corporation', 'EntityName': 'PEARTREE CLEANING SERVICES INC', 'EntityAddr': '15 SYDNEY DR, HYANNIS, MA 02601', 'AgentName': 'NELSIMAR PEREIRA', 'AgentAddr': '15 SYDNEY DR, HYANNIS, MA 02601'}>


shells [ArangoDoc '_id: corp_data_processed/17778062, _key: 17778062, _rev: _eZCy3Xa--E': <store: {'DataID': '00v62r', 'EntityTypeDescriptor': 'Domestic Profit Corporation', 'EntityName': 'PEARTREE CLEANING SERVICES INC', 'EntityAddr': '15 SYDNEY DR, HYANNIS, MA 02601', 'AgentName': 'NELSIMAR PEREIRA', 'AgentAddr': '15 SYDNEY DR, HYANNIS, MA 02601'}>]


corp ArangoDoc '_id: corp_data_processed/17778063, _key: 17778063, _rev: _eZCy3Xa--F': <store: {'DataID': '00000r', 'EntityTypeDescriptor': 'Domestic Limited Liability Company (LLC)', 'EntityName': 'BACK 9 CLUB LLC THE', 'EntityAddr': '17 HERITAGE HILL DR, LAKEVILLE, MA 02347', 'AgentName': 

In [52]:
def in_vertices(shell,shells):
    for i in shells:
        if shell._key == i._key:
            return True
    return False

In [51]:
print(shells[0]._id)

corp_data_processed/17778000
